<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת תשריטי תיעוד גבולות לבנק"ל מודרני 🗺️ 

In [ ]:
print('5. Tatag borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  5.a Reading inputs')

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData/"

today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תת"ג קיימת

In [ ]:
tatag_cols = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS', 'SURVEYOR_NO', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

tatag_layer = gpd.read_file(RawData_folder + 'CADSDE\TATAG.gdb',
                            layer = 'TATAG',
                            include_fields = tatag_cols)

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תב"ע קיימת

In [ ]:
taba_cols = ['PLAN','TAARICH','geometry']

taba_layer = gpd.read_file(RawData_folder + 'CADSDE\MMI_KAV_KACHOLRETZEF.gdb',
                           layer = 'MMI_KAV_KACHOLRETZEF',
                           include_fields = taba_cols)

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path, layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del [RawData_folder, today, date_name, tatag_cols, taba_cols, Blocks_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  5.b Cleaning data')

In [ ]:
print('  5.c Data conversion')

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני תב"ע
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:100%">
הגדרת שדה תאריך של תב"ע
<h1 style="text-align:right;font-size:100%">
סידור רשומות לפי תאריך
<h1 style="text-align:right;font-size:100%">
הסרת רשומות כפולות    
<h1 style="text-align:right;font-size:100%">
סינון תב"עות שנמצאות רק בשטחי תת"ג על ידי איחוד מרחבי   
<h1 style="text-align:right;font-size:100%">
הוספת שדה שם תהליך לצורך איחוד בהמשך

In [ ]:
taba_layer['TAARICH'] = pd.to_datetime(taba_layer['TAARICH'], dayfirst=True)

taba_layer.sort_values('TAARICH', inplace=True)

taba_layer = taba_layer.drop_duplicates(subset='PLAN',ignore_index=True, keep='last')

taba_layer = gpd.sjoin(tatag_layer, taba_layer, how='inner', predicate='intersects')

taba_layer['ProcessName'] = taba_layer['TATAG_NUM'].astype('str') + '/' + taba_layer['TATAG_YEAR'].astype('str')

taba_layer.sort_values(['ProcessName', 'TAARICH'], inplace=True)

taba_layer = taba_layer.drop_duplicates(subset = 'ProcessName', keep='last')

taba_layer = taba_layer[['ProcessName', 'PLAN']]
taba_layer.rename(columns={'PLAN':'PlanName'}, inplace=True)

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני גושים
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:100%">
הוספת שדה מקשר של שם מלא לגוש לטובת איחוד בהמשך

In [ ]:
Blocks = Blocks.astype(int)
Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני תת"ג
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:100%">
הסרת מספרי מלבן שאינם תקינים
<h1 style="text-align:right;font-size:100%">
הסרת רשומות ללא מידע של גוש    
<h1 style="text-align:right;font-size:100%">
מילוי ערכי מספרי תת-גוש ב-0    
<h1 style="text-align:right;font-size:100%">
הוספת שדה מקשר של שם מלא לגוש לטובת איחוד עם שכבת גושים וקבלת מספר מזהה לגוש

In [ ]:
legit_rectangle_min = 1
legit_rectangle_max = 4000

tatag_layer = tatag_layer[tatag_layer['TATAG_NUM'].between(legit_rectangle_min, legit_rectangle_max)]

del [legit_rectangle_min, legit_rectangle_max]

tatag_layer.dropna(subset = ['GUSH_NUM'], inplace=True)

tatag_layer['GUSH_NUM'] = tatag_layer['GUSH_NUM'].astype(int)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].fillna(0)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].astype(int)
tatag_layer['BlockName'] = tatag_layer['GUSH_NUM'].astype(str) + '_' + tatag_layer['GUSH_SUFFIX'].astype(str)

In [ ]:
tatag_layer.dropna(subset = ['GUSH_NUM'], inplace=True)

tatag_layer['GUSH_NUM'] = tatag_layer['GUSH_NUM'].astype(int)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].fillna(0)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].astype(int)
tatag_layer['BlockName'] = tatag_layer['GUSH_NUM'].astype(str) + '_' + tatag_layer['GUSH_SUFFIX'].astype(str)

tatag_layer = tatag_layer.merge(Blocks, on = 'BlockName', how='left')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  5.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                       #("BlockUniqueID", int), יאוחד בהמשך התהליך
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                       #("PlanName", str), יאוחד בהמשך התהליך
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", str)
                        ])


tatag_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                             geometry = 'geometry',
                             crs = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
tatag_new['MapYear'] = tatag_layer['TATAG_YEAR']


tatag_new['ProcessName'] = tatag_layer['TATAG_NUM'].astype('str') +'/'+ tatag_layer['TATAG_YEAR'].astype('str')


tatag_new['geometry'] = tatag_layer['geometry']


tatag_new['ProcessType'] = 4


tatag_new['DataSource'] = 5


statusDict = {0 : 21,  #הוגש
              1 : 22,  #בביקורת
              2 : 23}  #מאושר


tatag_new['Status'] = tatag_layer['TATAG_STATUS'].map(statusDict)


geodeticNetDict = {range(0, 1998)     : 1,     # רשת ישראל הישנה
                   range(1999, 2007)  : 2,     # רשת ישראל החדשה
                   range(2008, 99999) : 3}     # רשת ישראל התקפה
tatag_new['GeodeticNetwork'] = tatag_new['MapYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))


tatag_new['SurveyorLicenseID'] = tatag_layer['SURVEYOR_NO']
tatag_new['SurveyorLicenseID'] = tatag_new['SurveyorLicenseID'].astype(int)


tatag_new['BlockUniqueID'] = tatag_layer['BlockUniqueID']
tatag_new['BlockUniqueID'] = tatag_new['BlockUniqueID'].fillna(0)
tatag_new['BlockUniqueID'] = tatag_new['BlockUniqueID'].astype(int)


tatag_new = tatag_new.merge(taba_layer[['ProcessName', 'PlanName']], on ='ProcessName', how = 'left')
tatag_new = tatag_new.drop_duplicates(subset='ProcessName',ignore_index=True, keep='last')


#הסרת תכניות בעלי סטאטוס לא ידוע
tatag_new.dropna(inplace=True, subset=['Status'])


tatag_new['CPBUniqueID'] = range(1, 1 + len(tatag_new))
tatag_new['CPBUniqueID'] = tatag_new['CPBUniqueID'] + 4000000

In [ ]:
tatag_new.sort_values(['CPBUniqueID', 'MapYear'],inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  5.e Exporting results to product folder')

In [ ]:
#csv_dir  = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Tatag.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Tatag.json'


#tatag_new.drop(columns='geometry').to_csv(csv_dir ,encoding='cp1255', index=False)
tatag_new.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Tatag-Save as Feature Class.bat"]);